# 02 - Expériences Randomisées

## La Norme d'Or

Lors de la session précédente, nous avons vu pourquoi et comment l'association est différente de la causalité. Nous avons également vu ce qui est nécessaire pour que l'association soit la causalité.

$
E[Y|T=1] - E[Y|T=0] = \underbrace{E[Y_1 - Y_0|T=1]}_{ATT} + \underbrace{\{ E[Y_0|T=1] - E[Y_0|T=0] \}}_{BIAS}
$

Pour récapituler, l'association devient causalité s'il n'y a pas de biais. Il n'y aura pas de biais si $E[Y_0|T=0]=E[Y_0|T=1]$. En d'autres termes, l'association sera la causalité si les traités et les contrôles sont égaux ou comparables, à l'exception de leur traitement. Ou, en termes plus techniques, lorsque le résultat des non-traités est égal au résultat contrefactuel des traités. Rappelez-vous que ce résultat contrefactuel est le résultat du groupe traité s'il n'avait pas reçu le traitement.

Je pense que nous avons fait un bon travail pour expliquer comment rendre l'association égale à la causalité en termes mathématiques. Mais ce n'était qu'en théorie. Maintenant, nous examinons le premier outil que nous avons pour faire disparaître le biais : **les expériences randomisées**. Les expériences randomisées assignent aléatoirement des individus dans une population à un groupe de traitement ou à un groupe de contrôle. La proportion qui reçoit le traitement n'a pas besoin d'être de 50 %. Vous pourriez avoir une expérience où seulement 10 % de vos échantillons reçoivent le traitement.

La randomisation annihile le biais en rendant les résultats potentiels indépendants du traitement.

$
(Y_0, Y_1) \perp\!\!\!\perp T
$

Cela peut être déroutant au début (ça l'a été pour moi). Mais ne vous inquiétez pas, mon brave et vrai camarade, je vais vous l'expliquer plus en détail. Si le résultat est indépendant du traitement, cela n'implique-t-il pas également que le traitement n'a aucun effet ? Eh bien, oui ! Mais remarquez que je ne parle pas des résultats. Au lieu de cela, je parle des **résultats potentiels**. Le résultat potentiel est ce que le résultat **aurait été** sous traitement ($Y_1$) ou sous contrôle ($Y_0$). Dans les essais randomisés, nous **ne voulons pas** que le résultat soit **indépendant** du traitement puisque nous pensons que **le traitement cause** le résultat. Mais nous voulons que les **résultats potentiels** soient indépendants du traitement.

![img](./data/img/rct/indep.png)

Dire que les résultats potentiels sont indépendants du traitement revient à dire qu'ils seraient, en moyenne, les mêmes dans le groupe de traitement ou dans le groupe de contrôle. En termes plus simples, cela signifie que les groupes de traitement et de contrôle sont comparables. Ou que connaître l'attribution du traitement ne me donne aucune information sur le résultat avant le traitement. Par conséquent, $(Y_0, Y_1) \perp T$ signifie que le traitement est la seule chose générant une différence entre le résultat dans le groupe traité et dans le groupe de contrôle. Pour voir cela, remarquez que l'indépendance implique précisément que

$
E[Y_0|T=0]=E[Y_0|T=1]=E[Y_0]
$

Ce qui, comme nous l'avons vu, fait que

$
E[Y|T=1] - E[Y|T=0] = E[Y_1 - Y_0]=ATE
$

Ainsi, la randomisation nous donne un moyen d'utiliser une simple différence de moyennes entre le traitement et le contrôle et de l'appeler l'effet du traitement.

## Dans une École Très, Très Lointaine

En 2020, la pandémie de Coronavirus a forcé les entreprises à s'adapter à la distanciation sociale. Les services de livraison sont devenus répandus et les grandes entreprises ont adopté une stratégie de travail à distance. Pour les écoles, ce n'était pas différent. Beaucoup ont commencé leur propre dépôt en ligne de cours.

Quatre mois après le début de la crise, beaucoup se demandent si les changements introduits pourraient être maintenus. Il ne fait aucun doute que l'apprentissage en ligne présente des avantages. Il est moins cher car il permet d'économiser sur l'immobilier et le transport. Il peut également être plus numérique, tirant parti de contenus de classe mondiale du monde entier, et pas seulement d'un ensemble fixe d'enseignants. Malgré tout cela, nous devons encore répondre à la question de savoir si l'apprentissage en ligne a un impact négatif ou positif sur la performance académique des étudiants.

Une façon de répondre à cette question est de prendre des étudiants de écoles qui donnent principalement des cours en ligne et de les comparer avec des étudiants de écoles qui donnent des cours en présentiel. Comme nous le savons maintenant, ce n'est pas la meilleure approche. Il se pourrait que les écoles en ligne attirent uniquement les étudiants bien disciplinés qui réussissent mieux que la moyenne, même si le cours était en présentiel. Dans ce cas, nous aurions un biais positif, où les traités sont académiquement meilleurs que les non-traités : $E[Y_0|T=1] > E[Y_0|T=0]$.

Ou à l'inverse, il se pourrait que les cours en ligne soient moins chers et soient principalement composés d'étudiants moins riches, qui pourraient devoir travailler en plus d'étudier. Dans ce cas, ces étudiants feraient pire que ceux des écoles en présentiel, même s'ils prenaient des cours en présentiel. Si c'était le cas, nous aurions un biais dans l'autre direction, où les traités sont académiquement pires que les non-traités : $E[Y_0|T=1] < E[Y_0|T=0]$.

Donc, bien que nous puissions faire des comparaisons simples, ce ne serait pas convaincant. D'une manière ou d'une autre, nous ne pourrions jamais être sûrs qu'il n'y avait pas de biais caché qui masquait notre effet causal.

![img](./data/img/rct/lurking_bias.png)

Pour résoudre ce problème, nous devons rendre les traités et les non-traités comparables $E[Y_0|T=1] = E[Y_0|T=0]$. Une façon de forcer cela est d'assigner aléatoirement les cours en ligne et en présentiel aux étudiants. Si nous parvenions à le faire, les traités et les non-traités seraient, en moyenne, les mêmes, à l'exception du traitement qu'ils reçoivent.

Heureusement, certains économistes ont fait cela pour nous. Ils ont randomisé les cours de sorte que certains étudiants ont été assignés à avoir des cours en présentiel, d'autres à avoir uniquement des cours en ligne, et un troisième groupe à avoir un format mixte de cours en ligne et en présentiel. Ils ont collecté des données sur un examen standard à la fin du semestre.

Voici à quoi ressemblent les données :

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("./data/online_classroom.csv")
print(data.shape)
data.head()

(323, 10)


,gender,asian,black,hawaiian,hispanic,unknown,white,format_ol,format_blended,falsexam
0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,63.29997
1,1,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,79.96000
2,1,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,83.37000
3,1,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,90.01994
4,1,0.0,0.0,0.0,0.0,0.0,1.0,1,0.0,83.30000


Nous pouvons voir que nous avons 323 échantillons. Ce n’est pas exactement du big data, mais c’est quelque chose avec lequel nous pouvons travailler. Pour estimer l’effet causal, nous pouvons simplement calculer la moyenne des scores pour chacun des groupes de traitement.

In [4]:
(data
 .assign(class_format = np.select(
     [data["format_ol"].astype(bool), data["format_blended"].astype(bool)],
     ["online", "blended"],
     default="face_to_face"
 ))
 .groupby(["class_format"])
 .mean())

,gender,asian,black,hawaiian,hispanic,unknown,white,format_ol,format_blended,falsexam
class_format,,,,,,,,,,
blended,0.550459,0.217949,0.102564,0.025641,0.012821,0.012821,0.628205,0.0,1.0,77.093731
face_to_face,0.633333,0.202020,0.070707,0.000000,0.010101,0.000000,0.717172,0.0,0.0,78.547485
online,0.542553,0.228571,0.028571,0.014286,0.028571,0.000000,0.700000,1.0,0.0,73.635263


Oui. C'est aussi simple que cela. Nous pouvons voir que les cours en présentiel obtiennent un score moyen de 78,54, tandis que les cours en ligne obtiennent un score moyen de 73,63. Pas de très bonnes nouvelles pour les partisans de l'apprentissage en ligne. L'ATE pour un cours en ligne est donc de -4,91. Cela signifie que **les cours en ligne font en sorte que les étudiants obtiennent environ 5 points de moins**, en moyenne. C'est tout. Vous n'avez pas à vous inquiéter que les cours en ligne puissent avoir des étudiants plus pauvres qui ne peuvent pas se permettre des cours en présentiel ou, d'ailleurs, vous n'avez pas à vous inquiéter que les étudiants des différents traitements soient différents de quelque manière que ce soit, autre que le traitement qu'ils ont reçu. Par conception, l'expérience randomisée est faite pour éliminer ces différences.

Pour cette raison, une bonne vérification de la validité de la randomisation (ou si vous examinez les bonnes données) consiste à vérifier si les traités sont égaux aux non-traités dans les variables pré-traitement. Nos données contiennent des informations sur le genre et l'ethnicité pour voir s'ils sont similaires entre les groupes. Nous pouvons dire qu'ils semblent assez similaires pour les variables `genre`, `asiatique`, `hispanique` et `blanc`. La variable `noir`, cependant, semble un peu différente. Cela attire l'attention sur ce qui se passe avec un petit ensemble de données. Même sous randomisation, il se peut que, par hasard, un groupe soit différent d'un autre. Dans les grands échantillons, cette différence tend à disparaître.

## L'Expérience Idéale

Les expériences randomisées ou les essais contrôlés randomisés (ECR) sont le moyen le plus fiable d'obtenir des effets causals. C'est une technique simple et extrêmement convaincante. Elle est si puissante que la plupart des pays l'exigent pour démontrer l'efficacité de nouveaux médicaments. Pour faire une analogie terrible, vous pouvez penser aux ECR comme Aang, de *Avatar: Le Dernier Maître de l'Air*, tandis que d'autres techniques sont plus comme Sokka. Sokka est cool et peut réaliser quelques astuces sympas ici et là, mais Aang peut maîtriser les quatre éléments et se connecter au monde spirituel. Pensez-y de cette façon, si nous le pouvions, les ECR seraient tout ce que nous ferions pour découvrir la causalité. Un ECR bien conçu est le rêve de tout scientifique.

![img](./data/img/rct/science_dream.png)

Malheureusement, ils tendent à être soit très chers, soit tout simplement contraires à l'éthique. Parfois, nous ne pouvons tout simplement pas contrôler le mécanisme d'attribution. Imaginez-vous en tant que médecin essayant d'estimer l'effet du tabagisme pendant la grossesse sur le poids du bébé à la naissance. Vous ne pouvez pas simplement forcer une portion aléatoire de mamans à fumer pendant la grossesse. Ou disons que vous travaillez pour une grande banque et que vous devez estimer l'impact de la ligne de crédit sur la fidélisation des clients. Il serait trop coûteux de donner des lignes de crédit aléatoires à vos clients. Ou que vous voulez comprendre l'impact de l'augmentation du salaire minimum sur le chômage. Vous ne pouvez pas simplement assigner des pays à avoir un salaire minimum ou un autre. Vous voyez le problème.

Nous verrons plus tard comment réduire le coût de la randomisation en utilisant la randomisation conditionnelle, mais il n'y a rien que nous puissions faire à propos des expériences contraires à l'éthique ou irréalisables. Néanmoins, chaque fois que nous traitons des questions causales, il vaut la peine de penser à l'**expérience idéale**. Demandez-vous toujours, si vous le pouviez, **quelle serait l'expérience parfaite que vous mèneriez pour découvrir cet effet causal ?** Cela tend à éclairer la manière dont nous pouvons découvrir l'effet causal même sans l'expérience idéale.

## Le Mécanisme d'Attribution

Dans une expérience randomisée, le mécanisme qui attribue les unités à un traitement ou à un autre est, eh bien, aléatoire. Comme nous le verrons plus tard, toutes les techniques d'inférence causale essaieront d'une manière ou d'une autre d'identifier les mécanismes d'attribution des traitements. Lorsque nous savons avec certitude comment ce mécanisme se comporte, l'inférence causale sera beaucoup plus confiante, même si le mécanisme d'attribution n'est pas aléatoire.

Malheureusement, le mécanisme d'attribution ne peut pas être découvert en regardant simplement les données. Par exemple, si vous avez un ensemble de données où l'enseignement supérieur est corrélé avec la richesse, vous ne pouvez pas savoir avec certitude lequel a causé l'autre en regardant simplement les données. Vous devrez utiliser vos connaissances sur le fonctionnement du monde pour argumenter en faveur d'un mécanisme d'attribution plausible : est-ce que les écoles éduquent les gens, les rendant plus productifs et les menant à des emplois mieux rémunérés. Ou, si vous êtes pessimiste à propos de l'éducation, vous pouvez dire que les écoles ne font rien pour augmenter la productivité, et que c'est juste une corrélation fallacieuse parce que seules les familles riches peuvent se permettre d'envoyer un enfant à un diplôme supérieur.

Dans les questions causales, nous pouvons généralement argumenter dans les deux sens : que X cause Y, ou qu'il s'agit d'une troisième variable Z qui cause à la fois X et Y, et donc la corrélation entre X et Y n'est que fallacieuse. Pour cette raison, connaître le mécanisme d'attribution conduit à une réponse causale beaucoup plus convaincante. C'est aussi ce qui rend l'inférence causale si excitante. Alors que le machine learning appliqué consiste généralement à appuyer sur quelques boutons dans le bon ordre, l'inférence causale appliquée vous oblige à réfléchir sérieusement au mécanisme générant ces données.

## Idées Clés

Nous avons examiné comment les expériences randomisées sont le moyen le plus simple et le plus efficace de découvrir l'impact causal. Elles le font en rendant les groupes de traitement et de contrôle comparables. Malheureusement, nous ne pouvons pas faire des expériences randomisées tout le temps, mais il est toujours utile de penser à quelle serait l'expérience idéale que nous ferions si nous le pouvions.

Quelqu'un qui connaît les statistiques pourrait protester en ce moment que je n'ai pas examiné la variance de mon estimation de l'effet causal. Comment puis-je savoir qu'une diminution de 4,91 points n'est pas due au hasard ? En d'autres termes, comment puis-je savoir si la différence est statistiquement significative ? Et ils auraient raison. Ne vous inquiétez pas. Je compte passer en revue certains concepts statistiques ensuite.

## Références

J'aime penser à ce livre entier comme un hommage à Joshua Angrist, Alberto Abadie et Christopher Walters pour leur incroyable cours d'économétrie. La plupart des idées ici sont tirées de leurs cours à l'American Economic Association. Les regarder est ce qui me maintient sain d'esprit pendant cette année difficile de 2020.
* [Cross-Section Econometrics](https://www.aeaweb.org/conference/cont-ed/2017-webcasts)
* [Mastering Mostly Harmless Econometrics](https://www.aeaweb.org/conference/cont-ed/2020-webcasts)

Je tiens également à référencer les livres incroyables d'Angrist. Ils m'ont montré que l'économétrie, ou 'Metrics' comme ils l'appellent, n'est pas seulement extrêmement utile mais aussi profondément amusante.

* [Mostly Harmless Econometrics](https://www.mostlyharmlesseconometrics.com/)
* [Mastering 'Metrics](https://www.masteringmetrics.com/)

Ma dernière référence est le livre de Miguel Hernan et Jamie Robins. Il a été mon compagnon fidèle dans les questions causales les plus épineuses que j'ai dû résoudre.

* [Causal Inference Book](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/)

Les données utilisées ici proviennent d'une étude de Alpert, William T., Kenneth A. Couch, et Oskar R. Harmon. 2016. ["A Randomized Assessment of Online Learning"](https://www.aeaweb.org/articles?id=10.1257/aer.p20161057). American Economic Review, 106 (5): 378-82.

![img](./data/img/poetry.png)

## Contribuer

Causal Inference for the Brave and True est un matériel open-source sur l'inférence causale, la statistique de la science. Son objectif est d'être accessible monétairement et intellectuellement. Il utilise uniquement des logiciels gratuits basés sur Python.
Si vous avez trouvé ce livre précieux et souhaitez le soutenir, veuillez vous rendre sur [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true). Si vous n'êtes pas prêt à contribuer financièrement, vous pouvez également aider en corrigeant les fautes de frappe, en suggérant des modifications ou en donnant votre avis sur les passages que vous n'avez pas compris. Rendez-vous sur le repo du livre et [ouvrez une issue](https://github.com/matheusfacure/python-causality-handbook/issues). Enfin, si vous avez aimé ce contenu, veuillez le partager avec d'autres personnes qui pourraient le trouver utile et lui donner une [étoile sur GitHub](https://github.com/matheusfacure/python-causality-handbook/stargazers).